<a href="https://colab.research.google.com/github/pgssanjana/micro_projects/blob/main/face_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install -q kaggle
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle datasets list


Saving kaggle.json to kaggle.json


In [9]:
!kaggle datasets download -d pramod722445/face-recognition-30


face-recognition-30.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
import zipfile
zf = "/content/face-recognition-30.zip"
target_dir = "/content/input/face-recognition-30/dataset"
zfile = zipfile.ZipFile(zf)
zfile.extractall(target_dir)

In [11]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib as mpl

# style your matplotlib
mpl.style.use("seaborn-darkgrid")
# run this block



In [12]:
from tqdm import tqdm

In [13]:
files=os.listdir("/content/input/face-recognition-30/dataset/dataset/")
files

['mohamed_ali',
 'pewdiepie',
 'dhoni',
 'angelina_jolie',
 'ronaldo',
 'arnold_schwarzenegger',
 'Jim_Parsons',
 'virat_kohli',
 'ROHIT_SHARMA',
 'Matt_LeBlanc',
 'brad_pitt',
 'Simon_Helberg',
 'scarlett_johansson',
 'Jennifer_Aniston',
 'Courteney_Cox',
 'Matthew_Perry',
 'aishwarya_rai',
 'sylvester_stallone',
 'bhuvan_bam',
 'Johnny_Galeck',
 'manoj_bajpayee',
 'messi',
 'David_Schwimmer',
 'Sachin_Tendulka',
 'random_person',
 'Lisa_Kudrow',
 'hardik_pandya',
 'Kunal_Nayya',
 'Pankaj_Tripathi',
 'suresh_raina']

In [14]:
image_array=[]  # it's a list later i will convert it to array
label_array=[]
path="/content/input/face-recognition-30/dataset/dataset/"
# loop through each sub-folder in train
for i in range(len(files)):
    # files in sub-folder
    file_sub=os.listdir(path+files[i])

    for k in tqdm(range(len(file_sub))):
        try:
            img=cv2.imread(path+files[i]+"/"+file_sub[k])
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(96,96))
            image_array.append(img)
            label_array.append(i)
        except:
            pass

100%|██████████| 319/319 [00:00<00:00, 1113.26it/s]


In [15]:
import gc
gc.collect()

316

In [16]:
image_array=np.array(image_array)/255.0
label_array=np.array(label_array)


In [17]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

In [18]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential,Model,load_model

In [19]:
len(files)


30

In [20]:
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
# from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input

In [21]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [22]:
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

In [23]:

model=Sequential()
# I will use MobileNetV2 as an pretrained model 
pretrained_model=MobileNetV2(input_shape=(96,96,3),include_top=False,
                                         weights="imagenet")
model.add(pretrained_model)
model.add(layers.GlobalAveragePooling2D())
# add dropout to increase accuracy by not overfitting
model.add(layers.Dropout(0.3))
# add dense layer as final output
model.add(layers.Dense(1))
model.summary()

9420800/9406464 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_96 (Functi  (None, 3, 3, 1280)       2257984   
 onal)                                                           
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
____________________________________________

In [24]:
model.compile(optimizer="adam",loss="mean_squared_error",metrics=["mae"])



In [25]:

ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(filepath=ckp_path,
                                                   monitor="val_mae",
                                                   mode="auto",
                                                   save_best_only=True,
                                                   save_weights_only=True)


In [26]:
# create a lr reducer which decrease learning rate when accuarcy does not increase
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(factor=0.9,monitor="val_mae",
                                             mode="auto",cooldown=0,
                                             patience=5,verbose=1,min_lr=1e-6)
# patience : wait till 5 epoch
# verbose : show accuracy every 1 epoch
# min_lr=minimum learning rate

In [27]:
import tensorflow as tf
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',
                              monitor='val_loss',
                             verbose=1, 
                             save_best_only=True),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]

In [28]:
EPOCHS=10
BATCH_SIZE=64

history=model.fit(X_train,
                 Y_train,
                 validation_data=(X_test,Y_test),
                 batch_size=BATCH_SIZE,
                 epochs=EPOCHS,
                 callbacks=my_callbacks
                  

                 )

Epoch 1/10
212/212 [==============================] - ETA: 0s - loss: 51.1510 - mae: 5.3421
Epoch 1: val_loss improved from inf to 649.70471, saving model to model.01-649.70.h5
212/212 [==============================] - 56s 166ms/step - loss: 51.1510 - mae: 5.3421 - val_loss: 649.7047 - val_mae: 24.2453
Epoch 2/10
211/212 [============================>.] - ETA: 0s - loss: 23.8368 - mae: 3.4541
Epoch 2: val_loss improved from 649.70471 to 546.77399, saving model to model.02-546.77.h5
212/212 [==============================] - 32s 153ms/step - loss: 23.8408 - mae: 3.4545 - val_loss: 546.7740 - val_mae: 21.6277
Epoch 3/10
211/212 [============================>.] - ETA: 0s - loss: 16.6373 - mae: 2.7962
Epoch 3: val_loss improved from 546.77399 to 288.38068, saving model to model.03-288.38.h5
212/212 [==============================] - 32s 153ms/step - loss: 16.6513 - mae: 2.7970 - val_loss: 288.3807 - val_mae: 15.3748
Epoch 4/10
211/212 [============================>.] - ETA: 0s - loss: 13.

In [29]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp5pyva8vk/assets


INFO:tensorflow:Assets written to: /tmp/tmp5pyva8vk/assets


In [30]:
prediction_val=model.predict(X_test,batch_size=BATCH_SIZE)


In [31]:
prediction_val[:20]

array([[28.96565 ],
       [25.42814 ],
       [35.463375],
       [27.532999],
       [34.846992],
       [30.893404],
       [18.967182],
       [35.332695],
       [35.630703],
       [37.123726],
       [32.909786],
       [36.625977],
       [35.747444],
       [34.826057],
       [36.365086],
       [33.828236],
       [20.563824],
       [35.24498 ],
       [34.70197 ],
       [35.042057]], dtype=float32)

In [32]:
Y_test[:20]


array([10,  3,  1,  4, 21, 11,  6, 24, 13, 24, 18, 19, 20, 24, 24, 21,  4,
       24, 23, 10])